# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip uninstall -y kaggle
!pip install kaggle --upgrade


Found existing installation: kaggle 1.7.4.5
Uninstalling kaggle-1.7.4.5:
  Successfully uninstalled kaggle-1.7.4.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 10.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mokshitacv","key":"80702f13fe612ab3a8c049c2976be7b9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jorgeguerrapires/retinal-oct-images-small-version

Dataset URL: https://www.kaggle.com/datasets/jorgeguerrapires/retinal-oct-images-small-version
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
  0% 0.00/61.6M [00:00<?, ?B/s]
100% 61.6M/61.6M [00:00<00:00, 868MB/s]


In [ ]:
!unzip -q retinal-oct-images-small-version.zip -d retinal-oct-images-small-version

In [ ]:
!ls retinal-oct-images-small-version

CNV-samples  DME-samples  Drusen-samples  Normal-samples


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('retinal-oct-images-small-version',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4289 images belonging to 4 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('retinal-oct-images-small-version',
                                            target_size = (128,128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4289 images belonging to 4 classes.


splitting to test and train

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    'retinal-oct-images-small-version',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    'retinal-oct-images-small-version',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 3433 images belonging to 4 classes.
Found 856 images belonging to 4 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128,128, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=3))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

third layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

fourth layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dropout(0.5))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = val_set, epochs = 25)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 97s 874ms/step - accuracy: 0.2699 - loss: 1.3851 - val_accuracy: 0.4346 - val_loss: 1.2643
Epoch 2/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 93s 863ms/step - accuracy: 0.4667 - loss: 1.1844 - val_accuracy: 0.5491 - val_loss: 1.0529
Epoch 3/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 93s 864ms/step - accuracy: 0.5760 - loss: 0.9831 - val_accuracy: 0.5841 - val_loss: 0.9973
Epoch 4/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 862ms/step - accuracy: 0.6179 - loss: 0.8967 - val_accuracy: 0.6577 - val_loss: 0.8695
Epoch 5/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 93s 863ms/step - accuracy: 0.6745 - loss: 0.8242 - val_accuracy: 0.6157 - val_loss: 0.9241
Epoch 6/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 93s 858ms/step - accuracy: 0.7062 - loss: 0.7560 - val_accuracy: 0.6402 - val_loss: 0.8879
Epoch 7/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 93s 860ms/step - accuracy: 0.6777 - loss: 0.7758 - val_accuracy: 0.6834 - val_loss: 0.7804
Epoch 8/25
108/108 ━━━━━━━━━━━━━━━━━━━━ 143s 874ms/step - accuracy: 0.7284 - loss:

\\## Part 4 - Making a single prediction

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

test_image = image.load_img('retinal-oct-images-small-version/Normal-samples/824.jpg', target_size=(128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0  # Normalize like training data

In [ ]:
result = cnn.predict(test_image)
predicted_class = np.argmax(result)

labels = list(training_set.class_indices.keys())
print("Predicted class:", labels[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Predicted class: Normal-samples


confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_true = val_set.classes
y_pred_probs = cnn.predict(val_set)
y_pred = np.argmax(y_pred_probs, axis=1)

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_true, y_pred))


27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 371ms/step
Confusion Matrix:
[[197  37  11   4]
 [  5 156   0  16]
 [ 25   9 165  13]
 [  0  11  12 195]]

Accuracy Score:
0.8329439252336449
